# dependencies

In [168]:
class_dict = {'car': 0, 'head': 1, 'face': 2, 'human': 3, 'carplate': 4}
# class x_center y_center width height

In [1]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import random
from tqdm import tqdm

import numpy as np
import cv2
import matplotlib.pyplot as plt

# процессинг

In [ ]:
len(os.listdir('test/images'))

855

In [ ]:
len(os.listdir('train/images'))

2002

In [175]:
#распределение объектов в train
pd.Series([file_name.split('_')[-1] for file_name in os.listdir('train/labels')]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [176]:
class_dict

{'car': 0, 'head': 1, 'face': 2, 'human': 3, 'carplate': 4}

In [177]:
unique_files = list(set(['_'.join(file_name.split('_')[:-1]) for file_name in os.listdir('train/labels')]))

In [178]:
for unfile in tqdm(unique_files):
    
    text_ = ''
    
    for file_name in os.listdir('train/labels'):
        
        if unfile in file_name:
    
            file = '_'.join(file_name.split('_')[:-1])
            label = file_name.split('_')[-1].split('..')[0]
            class_ = class_dict[label]
            
            with open('train/labels/'+file_name, 'r') as f:
                text = f.read()
            
            for x in text.split('\n')[:-1]:
                x_ = x.split(' ')
                x_[0] = str(class_)
                
                text_ += ' '.join(x_) + '\n'
                
    text_ = text_[:-1]
    
    with open('train/labels_processed/'+unfile+'.txt', 'w') as f:
        f.write(text_)

In [12]:
train_files = os.listdir('train/images')
train_files_small = [x for x in train_files if '_' in x.split('-')[0]]
train_files_large = [x for x in train_files if '_' not in x.split('-')[0]]
print(f'train: {len(train_files_small)}, {len(train_files_large)}')

test_files = os.listdir('test/images')
test_files_small = [x for x in test_files if '_' in x.split('-')[0]]
test_files_large = [x for x in test_files if '_' not in x.split('-')[0]]
print(f'test: {len(test_files_small)}, {len(test_files_large)}')

train: 246, 1756
test: 111, 744


# подготовка сабмишена

In [40]:
import pandas as pd
import numpy as np
import os

In [77]:
# Предыдущий лучший
# solution-best-yolov7-w6-custom-small-images-640-multiscale.csv
# solution-best-yolov7-w6-custom-large_images-1920-iou05.csv

In [63]:
# Текущий лучший
sub_small = pd.read_csv('solution-best-yolov7-w6-custom-small-images-640-multiscale.csv', sep=';')
sub_large = pd.read_csv('solution-best-yolov7-e6-custom-large-images-1920.csv', sep=';')

test_files = os.listdir('test/images')
test_files_small = [x for x in test_files if '_' in x.split('-')[0]]
test_files_large = [x for x in test_files if '_' not in x.split('-')[0]]
print(f'test: {len(test_files_small)}, {len(test_files_large)}')

test: 111, 744


In [64]:
sub_small['LabelName'] = 4

In [65]:
sub_large = sub_large[sub_large.ImageID.isin(test_files_large)]

sub_small = sub_small[sub_small.LabelName == 4]
sub_small = sub_small[sub_small.ImageID.isin(test_files_small)]

In [66]:
sub_small = sub_small.sort_values('Conf', ascending=False).drop_duplicates('ImageID', keep='first')

In [67]:
sub_small.Conf.min()

0.20752

In [68]:
sub_small

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
2477,0_26-11-2021_10-11-18_AM.jpg,4,0.987793,0.075819,0.178279,0.652985,0.731343
29903,0_25-11-2021_10-03-50_AM.jpg,4,0.981445,0.058632,0.164495,0.760737,0.861963
50151,0_24-11-2021_01-12-29_PM.jpg,4,0.981445,0.805883,0.913725,0.613822,0.719512
661,0_23-11-2021_09-28-25_PM.jpg,4,0.975586,0.780142,0.900710,0.579269,0.701219
47555,2_24-11-2021_10-14-19_AM.jpg,4,0.975098,0.801961,0.911765,0.560150,0.661654
...,...,...,...,...,...,...,...
43463,0_23-11-2021_05-55-18_PM.jpg,4,0.816406,0.388889,0.682540,0.942653,0.980287
13233,0_26-11-2021_08-41-30_AM.jpg,4,0.813965,0.367089,0.753165,0.591836,0.698979
24922,0_25-11-2021_01-59-46_AM.jpg,4,0.813965,0.551219,0.729269,0.802272,0.968182
49760,1_26-11-2021_07-47-11_AM.jpg,4,0.662109,0.861111,0.964815,0.547619,0.735119


In [69]:
sub_small.Conf = np.clip(sub_small.Conf, 0.8, 1)

In [70]:
sub_large = sub_large[sub_large.Conf > 0.25]

In [71]:
sub = pd.concat([sub_small, sub_large])

In [72]:
sub.to_csv('solution-best-yolov7-e6-custom-large_1920-small_640_multiscale-iou05-conf025.csv', index=False, sep=';')

In [73]:
sub[(sub.ImageID.isin(test_files_small))].ImageID.count()

111

In [74]:
sub[(sub.ImageID.isin(test_files_large))].ImageID.nunique()

744

In [75]:
sub[(sub.ImageID.isin(test_files_small))].LabelName.value_counts()

4    111
Name: LabelName, dtype: int64

In [76]:
sub[(sub.ImageID.isin(test_files_large))].LabelName.value_counts()

0    8000
3    2170
1    2030
2     409
4      26
Name: LabelName, dtype: int64